<a href="https://colab.research.google.com/github/pratapsimha1/Android-Application-18-things/blob/master/Capstone_with_p.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os, glob
import numpy as np
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix

# ------------------------------------------------------------------------------
# 1) Load dataset (upload if not present)
# ------------------------------------------------------------------------------
candidates = glob.glob("/content/*default*credit*card*.xls") + \
             glob.glob("/content/*default*of*credit*card*.xls") + \
             glob.glob("/content/*default_of_credit_card_clients*.xls")
candidates = sorted(set(candidates))

if len(candidates) == 0:
    print("No dataset found. Upload the Excel file.")
    uploaded = files.upload()
    if len(uploaded) == 0:
        raise FileNotFoundError("No file uploaded.")
    path = "/content/" + list(uploaded.keys())[0]
else:
    path = candidates[0]

print("Using dataset:", path)

# ------------------------------------------------------------------------------
# 2) Read Excel & basic prep
# ------------------------------------------------------------------------------
df = pd.read_excel(path, header=1)

if 'default payment next month' in df.columns:
    df.rename(columns={'default payment next month': 'default'}, inplace=True)

if 'ID' in df.columns:
    df.drop(columns=['ID'], inplace=True)

print("Dataset shape:", df.shape)

X = df.drop(columns=['default'])
y = df['default']

# ------------------------------------------------------------------------------
# 3) Train-test split
# ------------------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# ------------------------------------------------------------------------------
# 4) BASELINE logistic regression (simple scaling, no cleaning)
# ------------------------------------------------------------------------------
scaler_base = StandardScaler()
X_train_base = scaler_base.fit_transform(X_train)
X_test_base = scaler_base.transform(X_test)

baseline_model = LogisticRegression(
    max_iter=2000, class_weight='balanced', n_jobs=-1
)
baseline_model.fit(X_train_base, y_train)

y_prob_base = baseline_model.predict_proba(X_test_base)[:,1]
y_pred_base = baseline_model.predict(X_test_base)

print("\n===== BASELINE MODEL =====")
print("Accuracy:", accuracy_score(y_test, y_pred_base))
print("ROC-AUC:", roc_auc_score(y_test, y_prob_base))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_base))

# ------------------------------------------------------------------------------
# 5) Calculate p-values for baseline logistic regression (FIXED)
# ------------------------------------------------------------------------------

from scipy.stats import norm

# Convert scaled training data to NumPy array
X_np = X_train_base  # shape: (n_samples, n_features)

# Add intercept column
X_np = np.hstack([X_np, np.ones((X_np.shape[0], 1))])

# Predicted probabilities on training data
p = baseline_model.predict_proba(X_train_base)[:, 1]

# Weight matrix (diagonal)
W = np.diag(p * (1 - p))

# Fisher Information Matrix: Xᵀ W X
XTWX = X_np.T @ W @ X_np

# Variance–covariance matrix
cov_matrix = np.linalg.inv(XTWX)

# Standard errors
std_errors = np.sqrt(np.diag(cov_matrix))

# Coefficients (features + intercept)
coefficients = np.append(
    baseline_model.coef_[0],
    baseline_model.intercept_
)

# z-statistics
z_scores = coefficients / std_errors

# two-sided p-values
p_values = (2 * (1 - norm.cdf(np.abs(z_scores)))).round(6)

# Feature names (add intercept)
feature_names = list(X_train.columns) + ['intercept']

# Summary table
pval_summary = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients,
    'Std_Error': std_errors,
    'z_value': z_scores,
    'p_value': p_values
}).sort_values('p_value')

print("\n===== BASELINE LOGISTIC REGRESSION P-VALUES =====")
print(pval_summary)

print("\n===== STATISTICALLY SIGNIFICANT FEATURES (p < 0.05) =====")
print(pval_summary[pval_summary['p_value'] < 0.05])


# ------------------------------------------------------------------------------
# 5) IMPROVED PIPELINE — cleaning + clipping + scaling + encoding
# ------------------------------------------------------------------------------

# ---- 5a) Identify numeric columns
numeric_cols = X_train.select_dtypes(include=['number']).columns.tolist()
print("\nNumeric columns:", len(numeric_cols))

# ---- 5b) Replace inf with NaN
X_train[numeric_cols] = X_train[numeric_cols].replace([np.inf, -np.inf], np.nan)
X_test[numeric_cols]  = X_test[numeric_cols].replace([np.inf, -np.inf], np.nan)

# ---- 5c) Clip extreme outliers (0.1% – 99.9%)
for c in numeric_cols:
    lo = np.nanpercentile(X_train[c], 0.1)
    hi = np.nanpercentile(X_train[c], 99.9)
    if np.isfinite(lo) and np.isfinite(hi):
        X_train[c] = X_train[c].clip(lo, hi)
        X_test[c]  = X_test[c].clip(lo, hi)

# ---- 5d) Median imputation
for c in numeric_cols:
    med = X_train[c].median()
    if np.isnan(med):
        med = 0.0
    X_train[c].fillna(med, inplace=True)
    X_test[c].fillna(med, inplace=True)

# ---- 5e) Scale numeric features
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled  = X_test.copy()
X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test_scaled[numeric_cols]  = scaler.transform(X_test[numeric_cols])

# ---- 5f) One-hot encode categoricals
cat_cols = X_train_scaled.select_dtypes(include=['object','category']).columns.tolist()
X_train_final = pd.get_dummies(X_train_scaled, columns=cat_cols, drop_first=True)
X_test_final  = pd.get_dummies(X_test_scaled,  columns=cat_cols, drop_first=True)

# Align columns
X_train_final, X_test_final = X_train_final.align(
    X_test_final, join='left', axis=1, fill_value=0
)

print("Final feature matrix:", X_train_final.shape)

interaction_pairs = [
    ('LIMIT_BAL', 'PAY_0'),
    ('PAY_0', 'PAY_2'),
    ('BILL_AMT1', 'PAY_AMT1')
]

for v1, v2 in interaction_pairs:
    if v1 in X_train_final.columns and v2 in X_train_final.columns:
        inter_name = f'{v1}_X_{v2}'
        X_train_final[inter_name] = X_train_final[v1] * X_train_final[v2]
        X_test_final[inter_name]  = X_test_final[v1]  * X_test_final[v2]

print("Interaction terms added:", [f"{v1}_X_{v2}" for v1, v2 in interaction_pairs])

# ------------------------------------------------------------------------------
# 6) IMPROVED logistic regression
# ------------------------------------------------------------------------------
improved_model = LogisticRegression(
    max_iter=5000, class_weight='balanced', solver='lbfgs'
)
improved_model.fit(X_train_final, y_train)

y_prob_imp = improved_model.predict_proba(X_test_final)[:,1]
y_pred_imp = improved_model.predict(X_test_final)

print("\n===== IMPROVED MODEL =====")
print("Accuracy:", accuracy_score(y_test, y_pred_imp))
print("ROC-AUC:", roc_auc_score(y_test, y_prob_imp))
print("\nClassification Report:\n", classification_report(y_test, y_pred_imp))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_imp))

# ------------------------------------------------------------------------------
print("\nPipeline completed successfully.")


Using dataset: /content/default_of_credit_card_clients (1).xls
Dataset shape: (30000, 24)

===== BASELINE MODEL =====
Accuracy: 0.6837777777777778
ROC-AUC: 0.7158905401027409
Confusion Matrix:
 [[4903 2106]
 [ 740 1251]]

===== BASELINE LOGISTIC REGRESSION P-VALUES =====
      Feature  Coefficient  Std_Error    z_value   p_value
0   LIMIT_BAL    -0.118244   0.019036  -6.211501  0.000000
2   EDUCATION    -0.093588   0.015979  -5.857075  0.000000
5       PAY_0     0.583040   0.021363  27.291954  0.000000
17   PAY_AMT1    -0.139145   0.026812  -5.189622  0.000000
18   PAY_AMT2    -0.235756   0.035798  -6.585692  0.000000
23  intercept    -0.197903   0.015021 -13.174903  0.000000
3    MARRIAGE    -0.076462   0.016498  -4.634487  0.000004
11  BILL_AMT1    -0.279832   0.064786  -4.319331  0.000016
6       PAY_2     0.110623   0.026474   4.178497  0.000029
4         AGE     0.064490   0.016757   3.848575  0.000119
1         SEX    -0.053599   0.015023  -3.567771  0.000360
7       PAY_3     0.

/tmp/ipython-input-1648246100.py:155: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[c].fillna(med, inplace=True)
/tmp/ipython-input-1648246100.py:156: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.

Final feature matrix: (21000, 23)
Interaction terms added: ['LIMIT_BAL_X_PAY_0', 'PAY_0_X_PAY_2', 'BILL_AMT1_X_PAY_AMT1']

===== IMPROVED MODEL =====
Accuracy: 0.7344444444444445
ROC-AUC: 0.7169225059636677

Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.78      0.82      7009
           1       0.43      0.57      0.49      1991

    accuracy                           0.73      9000
   macro avg       0.65      0.68      0.65      9000
weighted avg       0.77      0.73      0.75      9000

Confusion Matrix:
 [[5473 1536]
 [ 854 1137]]

Pipeline completed successfully.


In [ ]:
# ===================== FEATURE ENGINEERING + RETRAIN (ONE BLOCK) =====================

import os, glob
import numpy as np
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix

# ------------------------------------------------------------------------------
# 1) Load dataset
# ------------------------------------------------------------------------------
candidates = glob.glob("/content/*default*credit*card*.xls") + \
             glob.glob("/content/*default*of*credit*card*.xls") + \
             glob.glob("/content/*default_of_credit_card_clients*.xls")

if len(candidates) == 0:
    uploaded = files.upload()
    path = "/content/" + list(uploaded.keys())[0]
else:
    path = candidates[0]

df = pd.read_excel(path, header=1)
df.rename(columns={'default payment next month': 'default'}, inplace=True)
df.drop(columns=['ID'], inplace=True)

# ------------------------------------------------------------------------------
# 2) FEATURE ENGINEERING (THIS IS THE KEY DIFFERENCE)
# ------------------------------------------------------------------------------

# ---- Utilization features
for i in range(1, 7):
    df[f'util_{i}'] = df[f'BILL_AMT{i}'] / (df['LIMIT_BAL'] + 1)

df['util_mean'] = df[[f'util_{i}' for i in range(1, 7)]].mean(axis=1)

# ---- Payment behavior features
for i in range(1, 7):
    df[f'pay_ratio_{i}'] = df[f'PAY_AMT{i}'] / (df[f'BILL_AMT{i}'] + 1)

df['pay_ratio_mean'] = df[[f'pay_ratio_{i}' for i in range(1, 7)]].mean(axis=1)

# ---- Delinquency summary features
pay_cols = ['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']
df['max_delay'] = df[pay_cols].max(axis=1)
df['num_late_months'] = (df[pay_cols] > 0).sum(axis=1)

# ------------------------------------------------------------------------------
# 3) Train-test split
# ------------------------------------------------------------------------------
X = df.drop(columns=['default'])
y = df['default']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# ------------------------------------------------------------------------------
# 4) Clean + clip + impute numeric features
# ------------------------------------------------------------------------------
numeric_cols = X_train.select_dtypes(include=['number']).columns.tolist()

# Replace inf
X_train[numeric_cols] = X_train[numeric_cols].replace([np.inf, -np.inf], np.nan)
X_test[numeric_cols]  = X_test[numeric_cols].replace([np.inf, -np.inf], np.nan)

# Clip extremes
for c in numeric_cols:
    lo = np.nanpercentile(X_train[c], 0.1)
    hi = np.nanpercentile(X_train[c], 99.9)
    if np.isfinite(lo) and np.isfinite(hi):
        X_train[c] = X_train[c].clip(lo, hi)
        X_test[c]  = X_test[c].clip(lo, hi)

# Median imputation
for c in numeric_cols:
    med = X_train[c].median()
    X_train[c] = X_train[c].fillna(med)
    X_test[c]  = X_test[c].fillna(med)

# ------------------------------------------------------------------------------
# 5) Scale numeric features
# ------------------------------------------------------------------------------
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled  = X_test.copy()
X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test_scaled[numeric_cols]  = scaler.transform(X_test[numeric_cols])

# ------------------------------------------------------------------------------
# 6) One-hot encode categorical features
# ------------------------------------------------------------------------------
cat_cols = X_train_scaled.select_dtypes(include=['object','category']).columns.tolist()
X_train_final = pd.get_dummies(X_train_scaled, columns=cat_cols, drop_first=True)
X_test_final  = pd.get_dummies(X_test_scaled,  columns=cat_cols, drop_first=True)

X_train_final, X_test_final = X_train_final.align(
    X_test_final, join='left', axis=1, fill_value=0
)

print("Final feature matrix:", X_train_final.shape)

# ------------------------------------------------------------------------------
# 7) Train logistic regression
# ------------------------------------------------------------------------------
model = LogisticRegression(
    max_iter=5000,
    class_weight='balanced',
    solver='lbfgs'
)
model.fit(X_train_final, y_train)

# ------------------------------------------------------------------------------
# 8) Evaluation
# ------------------------------------------------------------------------------
y_pred = model.predict(X_test_final)
y_prob = model.predict_proba(X_test_final)[:, 1]

print("\n===== MODEL WITH BEHAVIORAL FEATURES =====")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# ------------------------------------------------------------------------------
print("\nDone. ")


Final feature matrix: (21000, 39)

===== MODEL WITH BEHAVIORAL FEATURES =====
Accuracy: 0.7445555555555555
ROC-AUC: 0.7526497645740546

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.78      0.83      7009
           1       0.44      0.61      0.51      1991

    accuracy                           0.74      9000
   macro avg       0.66      0.70      0.67      9000
weighted avg       0.78      0.74      0.76      9000

Confusion Matrix:
 [[5490 1519]
 [ 780 1211]]

Done. 


In [ ]:
# ===================== LOGISTIC vs XGBOOST (ONE FAIR COMPARISON BLOCK) =====================

# Install XGBoost if not present (Colab-safe)
!pip install xgboost

import os, glob
import numpy as np
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
import xgboost as xgb

# ------------------------------------------------------------------------------
# 1) Load dataset
# ------------------------------------------------------------------------------
candidates = glob.glob("/content/*default*credit*card*.xls") + \
             glob.glob("/content/*default*of*credit*card*.xls") + \
             glob.glob("/content/*default_of_credit_card_clients*.xls")

if len(candidates) == 0:
    uploaded = files.upload()
    path = "/content/" + list(uploaded.keys())[0]
else:
    path = candidates[0]

df = pd.read_excel(path, header=1)
df.rename(columns={'default payment next month': 'default'}, inplace=True)
df.drop(columns=['ID'], inplace=True)

# ------------------------------------------------------------------------------
# 2) Feature engineering (behavioral features)
# ------------------------------------------------------------------------------
# Utilization
for i in range(1, 7):
    df[f'util_{i}'] = df[f'BILL_AMT{i}'] / (df['LIMIT_BAL'] + 1)
df['util_mean'] = df[[f'util_{i}' for i in range(1, 7)]].mean(axis=1)

# Payment ratios
for i in range(1, 7):
    df[f'pay_ratio_{i}'] = df[f'PAY_AMT{i}'] / (df[f'BILL_AMT{i}'] + 1)
df['pay_ratio_mean'] = df[[f'pay_ratio_{i}' for i in range(1, 7)]].mean(axis=1)

# Delinquency summaries
pay_cols = ['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']
df['max_delay'] = df[pay_cols].max(axis=1)
df['num_late_months'] = (df[pay_cols] > 0).sum(axis=1)

# ------------------------------------------------------------------------------
# 3) Train-test split
# ------------------------------------------------------------------------------
X = df.drop(columns=['default'])
y = df['default']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# ------------------------------------------------------------------------------
# 4) Clean numeric features (shared preprocessing)
# ------------------------------------------------------------------------------
numeric_cols = X_train.select_dtypes(include=['number']).columns.tolist()

# Replace inf
X_train[numeric_cols] = X_train[numeric_cols].replace([np.inf, -np.inf], np.nan)
X_test[numeric_cols]  = X_test[numeric_cols].replace([np.inf, -np.inf], np.nan)

# Clip outliers
for c in numeric_cols:
    lo = np.nanpercentile(X_train[c], 0.1)
    hi = np.nanpercentile(X_train[c], 99.9)
    X_train[c] = X_train[c].clip(lo, hi)
    X_test[c]  = X_test[c].clip(lo, hi)

# Median impute
for c in numeric_cols:
    med = X_train[c].median()
    X_train[c] = X_train[c].fillna(med)
    X_test[c]  = X_test[c].fillna(med)

# ------------------------------------------------------------------------------
# 5) Prepare data for LOGISTIC REGRESSION (scale + encode)
# ------------------------------------------------------------------------------
scaler = StandardScaler()
X_train_lr = X_train.copy()
X_test_lr  = X_test.copy()
X_train_lr[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test_lr[numeric_cols]  = scaler.transform(X_test[numeric_cols])

cat_cols = X_train_lr.select_dtypes(include=['object','category']).columns.tolist()
X_train_lr = pd.get_dummies(X_train_lr, columns=cat_cols, drop_first=True)
X_test_lr  = pd.get_dummies(X_test_lr,  columns=cat_cols, drop_first=True)
X_train_lr, X_test_lr = X_train_lr.align(X_test_lr, join='left', axis=1, fill_value=0)

# ------------------------------------------------------------------------------
# 6) Prepare data for XGBOOST (NO scaling needed, encode only)
# ------------------------------------------------------------------------------
X_train_xgb = pd.get_dummies(X_train, columns=cat_cols, drop_first=True)
X_test_xgb  = pd.get_dummies(X_test,  columns=cat_cols, drop_first=True)
X_train_xgb, X_test_xgb = X_train_xgb.align(X_test_xgb, join='left', axis=1, fill_value=0)

# ------------------------------------------------------------------------------
# 7) Train LOGISTIC REGRESSION
# ------------------------------------------------------------------------------
lr_model = LogisticRegression(
    max_iter=5000,
    class_weight='balanced',
    solver='lbfgs'
)
lr_model.fit(X_train_lr, y_train)

lr_prob = lr_model.predict_proba(X_test_lr)[:,1]
lr_pred = lr_model.predict(X_test_lr)

# ------------------------------------------------------------------------------
# 8) Train XGBOOST
# ------------------------------------------------------------------------------
xgb_model = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    eval_metric='auc',
    scale_pos_weight=(y_train.value_counts()[0] / y_train.value_counts()[1]),
    random_state=42
)

xgb_model.fit(X_train_xgb, y_train)

xgb_prob = xgb_model.predict_proba(X_test_xgb)[:,1]
xgb_pred = xgb_model.predict(X_test_xgb)

# ------------------------------------------------------------------------------
# 9) Compare results
# ------------------------------------------------------------------------------
print("\n==================== MODEL COMPARISON ====================")

print("\n--- Logistic Regression ---")
print("Accuracy:", accuracy_score(y_test, lr_pred))
print("ROC-AUC:", roc_auc_score(y_test, lr_prob))
print("Confusion Matrix:\n", confusion_matrix(y_test, lr_pred))

print("\n--- XGBoost ---")
print("Accuracy:", accuracy_score(y_test, xgb_pred))
print("ROC-AUC:", roc_auc_score(y_test, xgb_prob))
print("Confusion Matrix:\n", confusion_matrix(y_test, xgb_pred))

# ------------------------------------------------------------------------------
print("\nDone. Compare ROC-AUC first; then inspect recall/precision trade-offs.")



==================== MODEL COMPARISON ====================

--- Logistic Regression ---
Accuracy: 0.7445555555555555
ROC-AUC: 0.7526497645740546
Confusion Matrix:
 [[5490 1519]
 [ 780 1211]]

--- XGBoost ---
Accuracy: 0.755
ROC-AUC: 0.7811092991654054
Confusion Matrix:
 [[5542 1467]
 [ 738 1253]]

Done. Compare ROC-AUC first; then inspect recall/precision trade-offs.


In [ ]:
# ===================== LOGISTIC REGRESSION → SCORECARD =====================

import numpy as np
import pandas as pd

# ------------------------------------------------------------------------------
# 1) Scorecard configuration (industry standard)
# ------------------------------------------------------------------------------
PDO = 20                    # Points to Double Odds
BASE_SCORE = 600            # Score at reference odds
BASE_ODDS = 50              # Good : Bad odds at base score

# Scaling constants
FACTOR = PDO / np.log(2)
OFFSET = BASE_SCORE - FACTOR * np.log(BASE_ODDS)

print(f"Scorecard settings → BASE_SCORE={BASE_SCORE}, PDO={PDO}, BASE_ODDS={BASE_ODDS}")

# ------------------------------------------------------------------------------
# 2) Extract logistic regression parameters
# ------------------------------------------------------------------------------
coefs = pd.Series(model.coef_[0], index=X_train_final.columns)
intercept = model.intercept_[0]

# ------------------------------------------------------------------------------
# 3) Create scorecard table (points per variable)
# ------------------------------------------------------------------------------
scorecard = pd.DataFrame({
    'feature': coefs.index,
    'coefficient': coefs.values,
})

# Convert coefficients to points
scorecard['points'] = -FACTOR * scorecard['coefficient']

# Add intercept as base points
base_points = OFFSET - FACTOR * intercept

print(f"\nBase points (intercept contribution): {base_points:.2f}")

# Sort by absolute importance
scorecard['abs_points'] = scorecard['points'].abs()
scorecard = scorecard.sort_values('abs_points', ascending=False)

print("\nTop 20 scorecard features by point impact:")
display(scorecard[['feature','coefficient','points']].head(20))

# ------------------------------------------------------------------------------
# 4) Define scoring function for any dataset
# ------------------------------------------------------------------------------
def score_customers(X, scorecard_df, base_points):
    """
    X: feature matrix (same columns as X_train_final)
    scorecard_df: dataframe with feature + points
    base_points: intercept-based score
    """
    points_map = scorecard_df.set_index('feature')['points']
    scores = base_points + X.mul(points_map, axis=1).sum(axis=1)
    return scores

# ------------------------------------------------------------------------------
# 5) Score train and test samples
# ------------------------------------------------------------------------------
train_scores = score_customers(X_train_final, scorecard, base_points)
test_scores  = score_customers(X_test_final, scorecard, base_points)

print("\nScore distribution (test set):")
print(test_scores.describe())

# ------------------------------------------------------------------------------
# 6) Validate monotonic relationship: score vs default rate
# ------------------------------------------------------------------------------
score_df = pd.DataFrame({
    'score': test_scores,
    'default': y_test.values
})

score_df['score_bin'] = pd.qcut(score_df['score'], 10, duplicates='drop')

bin_summary = score_df.groupby('score_bin').agg(
    avg_score=('score','mean'),
    default_rate=('default','mean'),
    count=('default','count')
).reset_index()

print("\nScorecard validation (by decile):")
display(bin_summary)

# ------------------------------------------------------------------------------
# 7) Example: scores of 10 customer
# ------------------------------------------------------------------------------
for i in range(0,10):
  print("\nExample customer:",[i])
  print("Predicted PD:", model.predict_proba(X_test_final.iloc[[i]])[0,1])
  print("Credit Score:", test_scores.iloc[i])


Scorecard settings → BASE_SCORE=600, PDO=20, BASE_ODDS=50

Base points (intercept contribution): 493.79

Top 20 scorecard features by point impact:


,feature,coefficient,points
38,num_late_months,0.834498,-24.078514
37,max_delay,0.315884,-9.114483
27,util_5,-0.206163,5.948594
0,LIMIT_BAL,-0.193863,5.593714
7,PAY_3,-0.157418,4.542129
5,PAY_0,0.156887,-4.526804
18,PAY_AMT2,-0.154330,4.453024
15,BILL_AMT5,0.135390,-3.906543
10,PAY_6,-0.131378,3.790780
6,PAY_2,-0.119594,3.450749



Score distribution (test set):
count    9000.000000
mean      493.932513
std        28.938731
min       387.223966
25%       477.940751
50%       504.047410
75%       512.442428
max       615.975485
dtype: float64

Scorecard validation (by decile):


/tmp/ipython-input-2763068608.py:80: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bin_summary = score_df.groupby('score_bin').agg(


,score_bin,avg_score,default_rate,count
0,"(387.223, 450.807]",431.549151,0.626667,900
1,"(450.807, 467.802]",459.157680,0.441111,900
2,"(467.802, 486.471]",477.465659,0.272222,900
3,"(486.471, 499.449]",494.356166,0.192222,900
4,"(499.449, 504.047]",501.939638,0.155556,900
5,"(504.047, 507.465]",505.772679,0.143333,900
6,"(507.465, 510.626]",509.015908,0.123333,900
7,"(510.626, 514.503]",512.487218,0.117778,900
8,"(514.503, 520.886]",517.408446,0.086667,900
9,"(520.886, 615.975]",530.172588,0.053333,900



Example customer: [0]
Predicted PD: 0.2827313698702812
Credit Score: 513.9845076515952

Example customer: [1]
Predicted PD: 0.6055064927172574
Credit Score: 474.76005873093214

Example customer: [2]
Predicted PD: 0.21041753212241715
Credit Score: 525.2795770088569

Example customer: [3]
Predicted PD: 0.8084302561660413
Credit Score: 445.57780431935606

Example customer: [4]
Predicted PD: 0.31020918953327864
Credit Score: 510.1812258870659

Example customer: [5]
Predicted PD: 0.2138942001076795
Credit Score: 524.6793988186654

Example customer: [6]
Predicted PD: 0.22619707617512796
Credit Score: 522.6105941819345

Example customer: [7]
Predicted PD: 0.5163357851116235
Credit Score: 485.23680043424645

Example customer: [8]
Predicted PD: 0.883515142363244
Credit Score: 428.66062748413253

Example customer: [9]
Predicted PD: 0.38182899719145874
Credit Score: 501.02446245397294


In [ ]:
# ===================== FINAL ROBUST WOE SCORECARD (ERROR-FREE) =====================

import os, glob
import numpy as np
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# ------------------------------------------------------------------------------
# 1) Load dataset
# ------------------------------------------------------------------------------
candidates = glob.glob("/content/*default*credit*card*.xls") + \
             glob.glob("/content/*default*of*credit*card*.xls") + \
             glob.glob("/content/*default_of_credit_card_clients*.xls")

if len(candidates) == 0:
    uploaded = files.upload()
    path = "/content/" + list(uploaded.keys())[0]
else:
    path = candidates[0]

df = pd.read_excel(path, header=1)
df.rename(columns={'default payment next month': 'default'}, inplace=True)
df.drop(columns=['ID'], inplace=True)

# ------------------------------------------------------------------------------
# 2) Feature engineering (behavioral)
# ------------------------------------------------------------------------------
for i in range(1, 7):
    df[f'util_{i}'] = df[f'BILL_AMT{i}'] / (df['LIMIT_BAL'] + 1)
df['util_mean'] = df[[f'util_{i}' for i in range(1,7)]].mean(axis=1)

for i in range(1, 7):
    df[f'pay_ratio_{i}'] = df[f'PAY_AMT{i}'] / (df[f'BILL_AMT{i}'] + 1)
df['pay_ratio_mean'] = df[[f'pay_ratio_{i}' for i in range(1,7)]].mean(axis=1)

pay_cols = ['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']
df['max_delay'] = df[pay_cols].max(axis=1)
df['num_late_months'] = (df[pay_cols] > 0).sum(axis=1)

# ------------------------------------------------------------------------------
# 3) CRITICAL FIX — remove infinities BEFORE WOE
# ------------------------------------------------------------------------------
num_cols = df.select_dtypes(include=['number']).columns

# Replace ±inf with NaN
df[num_cols] = df[num_cols].replace([np.inf, -np.inf], np.nan)

# Cap extreme values (winsorization)
for c in num_cols:
    lo = np.nanpercentile(df[c], 0.1)
    hi = np.nanpercentile(df[c], 99.9)
    if np.isfinite(lo) and np.isfinite(hi):
        df[c] = df[c].clip(lo, hi)

# Median imputation
for c in num_cols:
    df[c] = df[c].fillna(df[c].median())

# ------------------------------------------------------------------------------
# 4) SAFE WOE / IV function (now guaranteed to work)
# ------------------------------------------------------------------------------
def woe_iv_safe(data, feature, target, bins=5):
    d = data[[feature, target]].copy()
    d[feature] = pd.to_numeric(d[feature], errors='coerce')

    # Quantile binning (now safe)
    d['bin'] = pd.qcut(d[feature], bins, duplicates='drop')

    grouped = d.groupby('bin', observed=True)[target]

    tbl = pd.DataFrame({
        'good': grouped.count() - grouped.sum(),
        'bad': grouped.sum()
    })

    tbl['good_dist'] = tbl['good'] / tbl['good'].sum()
    tbl['bad_dist']  = tbl['bad'] / tbl['bad'].sum()

    tbl['woe'] = np.log((tbl['good_dist'] + 1e-6) / (tbl['bad_dist'] + 1e-6))
    tbl['iv']  = (tbl['good_dist'] - tbl['bad_dist']) * tbl['woe']

    return tbl.reset_index(), tbl['iv'].sum()

# ------------------------------------------------------------------------------
# 5) Build WOE dataset
# ------------------------------------------------------------------------------
woe_features = [
    'PAY_0',
    'util_mean',
    'pay_ratio_mean',
    'max_delay',
    'num_late_months',
    'LIMIT_BAL',
    'EDUCATION',
    'AGE',
    'SEX',
    'MARRIAGE'
]

df_woe = df.copy()
iv_list = []

for f in woe_features:
    table, iv = woe_iv_safe(df_woe, f, 'default', bins=5)
    iv_list.append((f, iv))

    df_woe[f + '_woe'] = pd.cut(
        df_woe[f],
        bins=table['bin'].cat.categories
    ).map(table.set_index('bin')['woe'])

iv_df = pd.DataFrame(iv_list, columns=['Feature','IV']).sort_values('IV', ascending=False)

print("\n===== INFORMATION VALUE (IV) =====")
print(iv_df)

# ------------------------------------------------------------------------------
# 6) Train WOE logistic regression (scorecard)
# ------------------------------------------------------------------------------
woe_cols = [f + '_woe' for f in woe_features]
# Convert WOE columns to float before filling NaNs
X = df_woe[woe_cols].apply(lambda s: s.astype(float))
X = X.fillna(0.0)
y = df_woe['default']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

woe_model = LogisticRegression(max_iter=2000)
woe_model.fit(X_train, y_train)

y_prob = woe_model.predict_proba(X_test)[:,1]
print("\nWOE Scorecard ROC-AUC:", roc_auc_score(y_test, y_prob))

# ------------------------------------------------------------------------------
# 7) Convert to scorecard + 10 examples
# ------------------------------------------------------------------------------
PDO, BASE_SCORE, BASE_ODDS = 20, 600, 50
FACTOR = PDO / np.log(2)
OFFSET = BASE_SCORE - FACTOR * np.log(BASE_ODDS)

coefs = pd.Series(woe_model.coef_[0], index=woe_cols)
intercept = woe_model.intercept_[0]

points = -FACTOR * coefs
base_points = OFFSET - FACTOR * intercept

scores = base_points + X_test.mul(points, axis=1).sum(axis=1)

examples = pd.DataFrame({
    'credit_score': scores,
    'predicted_PD': y_prob,
    'actual_default': y_test.values
})

examples_10 = pd.concat([
    examples.sort_values('credit_score').head(5),
    examples.sort_values('credit_score', ascending=False).head(5)
]).reset_index(drop=True)

print("\n===== 10 CUSTOMER SCORECARD EXAMPLES =====")
print(examples_10)





===== INFORMATION VALUE (IV) =====
           Feature        IV
0            PAY_0  0.869374
3        max_delay  0.736098
4  num_late_months  0.620070
5        LIMIT_BAL  0.155311
1        util_mean  0.117655
2   pay_ratio_mean  0.098141
7              AGE  0.012575
9         MARRIAGE  0.005649
6        EDUCATION  0.001679
8              SEX  0.000000

WOE Scorecard ROC-AUC: 0.7547358031959912

===== 10 CUSTOMER SCORECARD EXAMPLES =====
   credit_score  predicted_PD  actual_default
0    442.812804      0.822834               0
1    443.409550      0.819799               1
2    443.438478      0.819651               1
3    443.987180      0.816823               1
4    444.016108      0.816673               1
5    565.884499      0.061245               0
6    565.868820      0.061276               0
7    563.512733      0.066145               1
8    563.512733      0.066145               0
9    563.390513      0.066407               0
